## Učitati skup podataka 

In [371]:
import pandas as pd
import numpy as np

In [372]:
bank=pd.read_csv('C:\\Users\\Dusica\\Downloads\\Bank Marketing\\bank-additional.txt',sep=';')

## Deskriptivna statistika skupa podataka 

In [373]:
bank.shape


(4119, 21)

In [374]:
bank.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m
count,4119.000000,4116.000000,4114.000000,4119.000000,4117.000000,4116.000000,4117.000000,4117.000000,4117.000000
mean,40.113620,256.869048,2.534273,960.422190,0.190430,0.084694,93.579540,-40.486358,4.858993
std,10.313362,254.763913,2.566673,191.922786,0.541904,1.563361,0.579442,4.639840,79.432456
min,18.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.635000
25%,32.000000,103.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.334000
50%,38.000000,181.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000
75%,47.000000,317.250000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000
max,88.000000,3643.000000,35.000000,999.000000,6.000000,1.400000,94.767000,1.266000,5099.100000


In [375]:
bank.select_dtypes(include=['object']).describe()

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,nr.employed,y
count,4119,4119,4119,4119,4119,4119,4119,4119,4119,4119,4118,4118
unique,12,4,8,3,3,3,2,10,5,3,12,2
top,admin.,married,university.degree,no,yes,no,cellular,may,thu,nonexistent,5228.1,no
freq,1012,2509,1264,3315,2175,3349,2652,1378,860,3523,1625,3667


In [376]:
pd.concat([bank.dtypes,bank.isnull().sum()],axis=1)

,0,1
age,int64,0
job,object,0
marital,object,0
education,object,0
default,object,0
housing,object,0
loan,object,0
contact,object,0
month,object,0
day_of_week,object,0


## Priprema skupa podataka
(tako da bude pogodan za prediktivno modelovanje)

Input(duration) should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

In [377]:
bank.drop(['duration'],axis=1,inplace=True)

### *Resavanje null vrednosti*

In [378]:
bank.dropna(subset=['y'],inplace=True)

In [379]:
#set(bank['nr.employed'])

In [380]:
pd.concat([bank['cons.price.idx'].describe(),bank['previous'].describe()],axis=1)

#moze mean jer std ima malu vrednost

,cons.price.idx,previous
count,4116.000000,4116.000000
mean,93.579707,0.190476
std,0.579413,0.541961
min,92.201000,0.000000
25%,93.075000,0.000000
50%,93.749000,0.000000
75%,93.994000,0.000000
max,94.767000,6.000000


In [381]:
mean_cons_price_idx=bank['cons.price.idx'].dropna().mean()
#mean_cons_price_idx=np.average(bank['cons.price.idx'].dropna())

In [382]:
mean_previous=bank['previous'].dropna().mean()


In [383]:
bank['cons.price.idx'].replace(np.nan,mean_cons_price_idx,inplace=True)

In [384]:
bank['previous'].replace(np.nan,mean_previous,inplace=True)

In [385]:
bank['nr.employed']=pd.to_numeric(bank['nr.employed'],downcast='float')

In [386]:
bank['nr.employed'].fillna(value=bank['nr.employed'].dropna().mean(),inplace=True)

In [387]:
bank['nr.employed'].describe()

count    4118.000000
mean     5166.518555
std        73.662903
min      4963.600098
25%      5099.100098
50%      5191.000000
75%      5228.100098
max      5228.100098
Name: nr.employed, dtype: float64

campaign	float64	5
emp.var.rate	float64	3
cons.conf.idx	float64	2
euribor3m	float64	2

In [388]:
pd.concat([bank['campaign'].describe(),bank['emp.var.rate'].describe(),bank['cons.conf.idx'].describe(),bank['euribor3m'].describe()],axis=1)

,campaign,emp.var.rate,cons.conf.idx,euribor3m
count,4113.000000,4115.000000,4116.000000,4116.000000
mean,2.533917,0.085152,-40.496501,3.621325
std,2.566883,1.563274,4.594520,1.733618
min,1.000000,-3.400000,-50.800000,0.635000
25%,1.000000,-1.800000,-42.700000,1.334000
50%,2.000000,1.100000,-41.800000,4.857000
75%,3.000000,1.400000,-36.400000,4.961000
max,35.000000,1.400000,-26.900000,5.045000


In [389]:
bank['campaign'].replace(np.nan,bank['campaign'].dropna().mean(),inplace=True)

In [390]:
bank['emp.var.rate'].replace(np.nan,bank['emp.var.rate'].dropna().mean(),inplace=True)

In [391]:
bank['cons.conf.idx'].replace(np.nan,bank['cons.conf.idx'].dropna().mean(),inplace=True)

In [393]:
bank['euribor3m'].replace(np.nan,bank['euribor3m'].dropna().mean(),inplace=True)

In [394]:
bank['y'] = np.where(bank['y']=='no', 0, 1)

eliminate categorical variables from sample

In [395]:
bank_without_cat=bank.select_dtypes(exclude=['object'])

## Podela skupa podataka na trening i test u odnosu 70:30

In [396]:
def split_train_test(bank, test_percentage, shuffle=True):
    df=bank.sample(frac=1)
    n=len(df)
    split=int(np.floor(n*test_percentage))
    test= df.iloc[:split,:]
    train=df.iloc[split:,:]
    return (train, test)

In [397]:
#from sklearn.model_selection import train_test_split
#drugi nacin, ali ista stvar;identicno radi

In [398]:
train,test=split_train_test(bank,0.3)


In [399]:
train.shape

(2883, 20)

In [400]:
test.shape

(1235, 20)

In [401]:
y_train=train['y']
#y_train=train.loc[:,'y']

In [402]:
y_train.shape

(2883,)

In [403]:
x_train=train.drop('y',axis=1).select_dtypes(exclude=['object'])

In [404]:
x_train.shape

(2883, 9)

In [405]:
x_test=test.drop('y',axis=1).select_dtypes(exclude=['object'])

In [406]:
y_test=test['y']

Kreirati minimalno 2 prediktivna modela (sa default parametrima) i oceniti gresku na test setu (minimum 2 mere evaluacije)

In [407]:
#x_train.day_of_week.unique()

In [408]:
from sklearn.linear_model import LogisticRegression

In [409]:
lr=LogisticRegression()

In [410]:
modelLR=lr.fit(x_train,y_train)

In [411]:
predictionsLR=modelLR.predict(x_test)

In [412]:
predictionsLR

array([0, 0, 0, ..., 0, 0, 0])

In [413]:
print(type(predictionsLR))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [414]:
y_real=np.array(y_test)

In [415]:
y_real_train=np.array(y_train)

In [416]:
y_real

array([0, 0, 0, ..., 0, 0, 1])

In [417]:
predictionsLR=modelLR.predict(x_test)


In [418]:
predict_train=modelLR.predict(x_train)


In [419]:
true_predicted=np.column_stack((y_real, predictionsLR))


In [420]:
prediction_probs=modelLR.predict_proba(x_test)

In [421]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

In [422]:
print(accuracy_score(y_train,predict_train)

0.90565383281304201

In [425]:
precision_score(y_train,predict_train)
recall_score(y_train,predict_train)

0.22756410256410256

In [424]:
accuracy_score(y_test,predictionsLR)

0.8963562753036437

In [55]:
confusion_matrix(y_test, predictionsLR)

array([[1073,   11],
       [ 121,   30]], dtype=int64)

In [56]:
y_test.value_counts()

no     1084
yes     151
Name: y, dtype: int64

In [57]:
precision_score(y_test,predictionsLR,pos_label='yes',average="binary")

0.73170731707317072

In [58]:
recall_score(y_test,predictionsLR,pos_label='yes')

0.19867549668874171

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
rf=RandomForestClassifier()

In [61]:
modelRF=rf.fit(x_train,y_train)

In [62]:
predictionsRF=modelRF.predict(x_test)

In [63]:
predictionsRF

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [64]:
accuracy_score(y_test,predictionsRF)

0.86396761133603239

In [65]:
confusion_matrix(y_test,predictionsRF)

array([[1035,   49],
       [ 119,   32]], dtype=int64)

In [66]:
precision_score(y_test,predictionsRF,pos_label='yes')

0.39506172839506171

In [67]:
recall_score(y_test,predictionsRF,pos_label='yes')

0.2119205298013245

In [292]:
from sklearn.neighbors import KNeighborsClassifier

In [294]:
knn=KNeighborsClassifier()

In [297]:
modelknn=knn.fit(x_train, y_train)

In [298]:
predictionsKNN=knn.predict(x_test)

In [299]:
prediction_probs_knn=knn.predict_proba(x_test)

In [300]:
accuracy_score(y_test,predictionsKNN)

0.88178137651821864

In [301]:
precision_score(y_test,predictionsKNN,pos_label='yes',average="binary")

0.53846153846153844

In [302]:
recall_score(y_test,predictionsKNN,pos_label='yes')

0.23178807947019867

### Cross validation

In [68]:
from sklearn.model_selection import cross_val_score

In [69]:
models=[lr,rf]

In [70]:
len(models)

2

In [73]:
for mod in models:
    print (mod)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [74]:
for x in range(len(models)):
     #print (x)
     print (type(models[-1]))

<class 'sklearn.ensemble.forest.RandomForestClassifier'>
<class 'sklearn.ensemble.forest.RandomForestClassifier'>


In [75]:
for num in range(len(models)):
    scores=cross_val_score(models[num-1],x_train,y_train,cv=5,scoring='accuracy')
    print (scores,'--->',scores.mean())

[ 0.88734835  0.88908146  0.89081456  0.87847222  0.88715278] ---> 0.886573873484
[ 0.89948007  0.90987868  0.90641248  0.90104167  0.91666667] ---> 0.906695912767


In [76]:
#np.average(scores)

## Optimizacija parametara

In [196]:
from sklearn.model_selection import GridSearchCV

In [197]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [222]:
params = [
  {'n_estimators': [10, 20, 40, 100, 300, 400], 'max_depth': [1, 5, 10, 15]}
 ]

In [223]:
optimizer = GridSearchCV(rf, 
                   param_grid=params, 
                         cv=3,  
                   return_train_score=True)

In [224]:
opt_rf=optimizer.fit(x_train,y_train)

In [304]:
opt_rf.best_estimator_


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [227]:
probas_opt=opt_rf.predict_proba(x_test)

In [228]:
predictions_opt=opt_rf.predict(x_test)


In [290]:
lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [303]:
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [305]:
params2 = [
  {'leaf_size': [10, 20, 40, 50, 100], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7]}
 ]


In [308]:
optimizer2 = GridSearchCV(knn, 
                   param_grid=params2, 
                         cv=3,  
                   return_train_score=True)

In [309]:
opt_rf2=optimizer2.fit(x_train,y_train)

In [310]:
opt_rf2.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='uniform')

In [311]:
probas_opt_knn=opt_rf2.predict_proba(x_test)
predictions_opt_knn=opt_rf2.predict(x_test)

In [427]:
cross_val_score(rf, x_train, y_train, cv=5, scoring='roc_auc')

array([ 0.7373247 ,  0.70905132,  0.69737668,  0.69022217,  0.66130601])

## Parameter optimization - Results
### Random Forest 

In [282]:
print('ACCURACY\n','Before',accuracy_score(y_test,predictionsRF),'\n After ',accuracy_score(y_test,predictions_opt))

ACCURACY
 Before 0.863967611336 
 After  0.879352226721


In [288]:
print('PRECISION\n Before',precision_score(y_test,predictionsRF,pos_label='yes',average="binary"),'\n After ',precision_score(y_test,predictions_opt,pos_label='yes',average="binary"))

PRECISION
 Before 0.395061728395 
 After  0.520833333333


In [289]:
print('RECALL\n Before',recall_score(y_test,predictionsRF,pos_label='yes'),'\n After ',recall_score(y_test,predictions_opt,pos_label='yes'))

RECALL
 Before 0.211920529801 
 After  0.165562913907


### KNN 

In [312]:
print('Before',accuracy_score(y_test,predictionsKNN),'\nAfter ',accuracy_score(y_test,predictions_opt_knn))

Before 0.881781376518 
After  0.881781376518


In [313]:
print('Before',precision_score(y_test,predictionsKNN,pos_label='yes',average="binary"),'\nAfter ',precision_score(y_test,predictions_opt_knn,pos_label='yes',average="binary"))

Before 0.538461538462 
After  0.558139534884


In [314]:
print('Before',recall_score(y_test,predictionsKNN,pos_label='yes'),'\nAfter ',recall_score(y_test,predictions_opt_knn,pos_label='yes'))

Before 0.23178807947 
After  0.158940397351


### Podela inicijalnog skupa podataka na osnovu jednog kategoričkog atributa u datasetu

In [96]:
#bank.sort_values(['contact'])

In [125]:
#bank.loc[bank['loan'] == 'yes']

In [126]:
loan_yes=bank.loc[lambda bank:bank['loan']=='yes']

In [315]:
categories=bank.loan.unique()

In [128]:
for n in range(len(categories)):
        print(n)

0
1
2


In [316]:
listaDF=[pd.DataFrame,pd.DataFrame,pd.DataFrame]

In [317]:
#def vratiSS():
for n in range(len(categories)):
    listaDF[n]=bank.loc[lambda bank:bank['loan']==categories[n]]
    #return listaDF[n]

In [183]:
#listaDF[1].loan.unique()


In [191]:
for rb in range(len(listaDF)):
    scores=cross_val_score(rf,listaDF[rb].drop('y',axis=1).select_dtypes(exclude=['object']),listaDF[rb]['y'],cv=5,scoring='accuracy')
    print (len(listaDF[rb].loan),listaDF[rb].loan.unique(),scores,'--->',scores.mean())
    

3349 ['no'] [ 0.87761194  0.88656716  0.88358209  0.86716418  0.87593423] ---> 0.878171920666
105 ['unknown'] [ 0.90909091  0.9047619   0.95238095  0.95238095  1.        ] ---> 0.943722943723
664 ['yes'] [ 0.85074627  0.87218045  0.87969925  0.89393939  0.88636364] ---> 0.876585799642


In [340]:
categories=bank.marital.unique()
listaDF=[pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame]
for n in range(len(categories)):
    #listaDF.append(pd.DataFrame)
    listaDF[n]=bank.loc[lambda bank:bank['marital']==categories[n]]
for rb in range(len(listaDF)):
    scores=cross_val_score(rf,listaDF[rb].drop('y',axis=1).select_dtypes(exclude=['object']),listaDF[rb]['y'],cv=5,scoring='accuracy')
    print(len(listaDF[rb].marital),listaDF[rb].marital.unique(),scores,'--->',scores.mean())

 

2508 ['married'] [ 0.89662028  0.89442231  0.89820359  0.9001996   0.89021956] ---> 0.895933068716
1153 ['single'] [ 0.86580087  0.80952381  0.84415584  0.87391304  0.85652174] ---> 0.849983060418
446 ['divorced'] [ 0.87777778  0.87777778  0.93333333  0.92045455  0.875     ] ---> 0.896868686869
11 ['unknown'] [ 0.66666667  1.          1.          1.          1.        ] ---> 0.933333333333


C:\Users\Dusica\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [344]:
from fancyimpute import KNN
#X is the complete data matrix
# X_incomplete has the same values as X except a subset have been replace with NaN

# Use 3 nearest rows which have a feature to fill in each row's missing features



ModuleNotFoundError: No module named 'cvxpy'

In [345]:
conda install -c https://conda.anaconda.org/omnia scs

SyntaxError: invalid syntax (<ipython-input-345-641a1183c183>, line 1)

## Recursive feature elimination

In [348]:
# Feature Extraction with RFE
from sklearn.feature_selection import RFE

#names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

#array = bank.values
#X = array[:,0:8]
#Y = array[:,8]

# feature extraction
rfe = RFE(lr, 3)
fit = rfe.fit(x_train, y_train)
#print("Num Features: %d") % fit.n_features_
#print("Selected Features: %s") % fit.support_
#print("Feature Ranking: %s") % fit.ranking_

In [355]:
fit.support_

array([False,  True, False,  True, False, False, False,  True, False], dtype=bool)

In [844]:
import seaborn as sns

In [851]:
ax1=sns.distplot(bank['y'],hist=False)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [760]:
type(np.array(prediction_probs))


numpy.ndarray

In [765]:
prediction_probs[1]

array([ 0.95796279,  0.04203721])

In [773]:
import numpy as np
import matplotlib.pyplot as plt

#t = np.arange(0., 1., 0.2)
x,y= np.polyfit(np.array(prediction_probs), np.array(prediction_probs), 1)
# red dashes, blue squares and green triangles
plt.plot( x, 'r--', y, 'bs')
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

TypeError: expected 1D vector for x